In [1]:
#Imports

from collections import Counter

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import time
import tqdm
import json
import re

In [2]:
# importing the module
import json
  
# Opening JSON file
f = open('json_datasets/train.json', 'r')

raw_train = json.load(f)

In [3]:
def removeQuotes(lst):
    ret = []
    for s in lst:
        if s != '``' and s != "''":
            ret.append(s)
    return ret

def camelCaseSplit(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    d = [m.group(0) for m in matches]
    new_d = []
    for token in d:
        token = token.replace('(', '')
        token = token.replace(')', '')
        token_split = token.split('_')
        for t in token_split:
            #new_d.append(t.lower())
            new_d.append(t)
    return new_d

In [4]:
def g2tPreprocess(raw):
    graphs = []
    texts = []
    for item in raw:
        for relation in item['relations']:
            graph = ' <H> ' + ' '.join(removeQuotes(relation[0])) + ' <R> '
            graph += ' '.join(camelCaseSplit(relation[1])) + ' <T> '
            graph += ' '.join(removeQuotes(relation[2]))
            graphs.append(graph)
        ents = [' '.join(removeQuotes(entity)) for entity in item['entities']]
        text = item['text']
        for i in range(len(ents)):
            text = text.replace('<ENT_'+str(i)+'>', ents[i])
        texts.append(text)
    return graphs, texts

In [5]:
graphs, texts = g2tPreprocess(raw_train)
print(graphs[:10])
print(texts[:10])

[' <H> Aarhus Airport <R> city Served <T> Aarhus , Denmark', ' <H> Aarhus Airport <R> city Served <T> Aarhus , Denmark', ' <H> Aarhus Airport <R> city Served <T> Aarhus', ' <H> Aarhus Airport <R> elevation Above The Sea Level in metres <T> 25.0', ' <H> Aarhus Airport <R> elevation Above The Sea Level in metres <T> 25.0', ' <H> Aarhus Airport <R> elevation Above The Sea Level in metres <T> 25.0', ' <H> Aarhus Airport <R> location <T> Tirstrup', ' <H> Aarhus Airport <R> location <T> Tirstrup', ' <H> Aarhus Airport <R> operating Organisation <T> Aarhus Lufthavn A/S', ' <H> Aarhus Airport <R> operating Organisation <T> Aarhus Lufthavn A/S']
['the Aarhus Airport of Aarhus , Denmark .', 'Aarhus Airport serves the city of Aarhus , Denmark .', 'Aarhus Airport serves the city of Aarhus .', 'Aarhus Airport is 25.0 metres above sea level .', 'Aarhus Airport is at an elevation of 25.0 metres above seal level .', 'Aarhus Airport is 25.0 metres above the sea level .', 'Aarhus Airport is located in T